<div style="border-radius:10px; padding: 15px; background-color:lavender; font-size:130%; text-align:left">

<h1 align="left"><font color=blue>Description: </font></h1>
    
<p style="color:black;">In this project, we aim to develop a sophisticated automated garbage classification system leveraging the ResNet50 pre-trained model architecture, utilizing the power of transfer learning in the Artificial Intelligence field. This notebook serves as the main entry point for the software of our automated garbage classification system. It connects the pre-trained model, the camera output, capacitive and inductive sensor outputs, motors and actuators to achieve the objectives of our project.</p>

</div>


<div style="border-radius:10px; padding: 15px; background-color:lavender; font-size:130%; text-align:left">

<h1 align="left"><font color=blue>Dataset: </font></h1>
    
<p style="color:black;">This project utilizes the "TrashBox dataset" for the ResNet50 model.
Thhis dataset contains 17785 waste object images divided into seven classes (glass, plastic, metal, e-waste, cardboard, paper, medical waste) for training. 

<p style="color:black;">The dataset contains 7 classes even though our project classifies trash into just 4 classes. The other classes, however, will not be removed but just ignored. This will allow for further future improvements if needed.  </p>

</div>


<div style="border-radius:10px; padding: 15px; background-color:lavender; font-size:130%; text-align:left">

<h1 align="left"><font color=blue>Objectives: </font></h1>
    
<ul style="color:black;">
    <li><strong>Employ Transfer Learning:</strong> Leverage a pre-trained ResNet50, adapting it for our specific dataset.</li>
    <li><strong>Accept sensor data:</strong> Accept and clean all sensor data including optical, capacitive and inductive.</li>
    <li><strong>Develop algorithm for predicting trash class:</strong> develop a multimodal approach leveraging all sensor data to make a prediction on the trash class.</li>
    <li><strong>Motors & Actuators integration:</strong> Send the prediction to actuators to place trash in the correct pin.</li>

</ul>

</div>



<a id="Initialization"></a>
# <p style="background-color: royalblue; font-family:calibri; color:white; font-size:140%; font-family:Verdana; text-align:center; border-radius:15px 50px;">Step 1 | Initialization</p>

# **Importing necessary libraries**

In [ ]:
import cv2 
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.models import load_model
import os
from PIL import Image
import time

Explanation:
1. import numpy

In [ ]:
# Function to validate and remove corrupted images
def validate_and_remove_corrupted_images(directory):
    for root, dirs, files in os.walk(directory): # search through all directories and files
        for file in files: # search each file 
            if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')): # specifically search for pictures 
                file_path = os.path.join(root, file) 
                try:
                    img = Image.open(file_path) # open the image file
                    img.verify()  # Verify that it is an image
                except (IOError, SyntaxError) as e: # if an exception pccurs
                    print(f"Removing bad file: {file_path} - {e}") 
                    os.remove(file_path) # remove corrupted image from the dataset


In [ ]:
# Validate and remove corrupted images from your training and validation sets
validate_and_remove_corrupted_images('/Users/macbook/Desktop/TrashBox-main/TrashBox_train_set') 
validate_and_remove_corrupted_images('/Users/macbook/Desktop/TrashBox-testandvalid-main/TrashBox_testandvalid_set')


In [ ]:
# Load the pre-trained EfficientNet model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) 
x = base_model.output 
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(7, activation='softmax')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Load the TrashBox dataset and fine-tune the model
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    '/Users/macbook/Desktop/TrashBox-main/TrashBox_train_set', # train set path, change if run on different computer. 
    target_size=(224, 224), # image size 
    batch_size=32, # batch size 
    class_mode='categorical',  # categorical since we have 6 classes 
    shuffle=True  # Shuffle training data for better training
)

In [ ]:
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    '/Users/macbook/Desktop/TrashBox-testandvalid-main/TrashBox_testandvalid_set/val',  # Point to 'val' subdirectory, change if run on different computer accordingly
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  
    shuffle=False  # Don't shuffle validation data to keep it consistent 
)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy']) # compile model with adam optimizer 


In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20
)


In [ ]:
# Save the model after training
model.save('trash_classification_model.h5')

In [ ]:
# Test the model using the 'test' dataset
test_generator = validation_datagen.flow_from_directory(
    '/Users/macbook/Desktop/TrashBox-testandvalid-main/TrashBox_testandvalid_set/test',  # Point to 'test' subdirectory, change if run on different computer accordingly. 
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Accuracy: {test_accuracy:.2f}, Test Loss: {test_loss:.2f}")

In [ ]:
# Load the pre-trained model
model = load_model('/Users/macbook/Desktop/trash_classification_model.h5')

# Class labels 
class_labels = {
    0: 'cardboard',
    1: 'e-waste',
    2: 'glass',
    3: 'medical',
    4: 'metal',
    5: 'paper',
    6: 'plastic'
}

In [ ]:
# Initialize webcam for real-time classification
cap = cv2.VideoCapture(0) 
last_prediction_time = time.time()
last_label = ""  # To store the last prediction label, initially empty
last_probabilities = None  # To store the last prediction probabilities, initially None

In [ ]:
while True: # infinite loop, to run until stopped. 
    ret, frame = cap.read()
    if not ret:
        break

    # to get a prediction every 10 seconds, change and tune if needed. because 1 second is too little time to observe the results
    # Check if 10 seconds have passed since the last prediction
    current_time = time.time()
    if current_time - last_prediction_time >= 10:
        # Preprocess the frame for EfficientNet
        img = cv2.resize(frame, (224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        # Predict the class
        predictions = model.predict(img_array)
        predicted_class = np.argmax(predictions, axis=1)[0]
        last_label = class_labels[predicted_class]  # Update the last prediction label
        last_probabilities = predictions[0]  # Update the last prediction probabilities
        last_prediction_time = current_time  # Update the last prediction time

    # Display the last predicted label on the frame
    if last_label:
        cv2.putText(frame, f'Prediction: {last_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the probabilistic output of the last prediction on the frame
    if last_probabilities is not None:
        for i, (label, prob) in enumerate(zip(class_labels.values(), last_probabilities)):
            text = f'{label}: {prob:.2f}'
            cv2.putText(frame, text, (10, 60 + i * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)

    # Display the frame with the label and probabilities
    cv2.imshow('Trash Classification', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release() # for security and efficiency
cv2.destroyAllWindows() # for security and efficiency
